In [1]:
using Optim
using DataFrames;using CSV
using Pkg
using LinearAlgebra;
using JuliaZH;JuliaZH.set_mirror("PKU")#JuliaZH.generate_startup("BFSU")
ccall((:openblas_get_num_threads64_, Base.libblas_name),Cint,())
LinearAlgebra.BLAS.set_num_threads(8)
using Base.Threads;nthreads()
using Meteor.ExactDiag;
using JLD2;using FileIO
using DifferentialEquations

In [2]:
function dm2vec(dm)
    return reshape(dm,:,1)
end

function vec2dm(vec)
    return reshape(vec,3,:)
end
function thermal_state(dims,nbar)
    c::Array{Complex{Float64}}=zeros(dims,dims)
    if nbar==0
        c[1,1]=1
    else
        for i in 1:dims
           c[i,i]= exp(-i*log(1/nbar+1))
        end
    end
    c=c/tr(c)
    return c        
end
    
function coherence_state(alpha)
    m=exp(alpha*a₊(dim)-alpha'*a₋(dim))*n0(dim)
    return  m*m'
end
    
function fock_state(dims,n)
    c::Matrix{ComplexF64}=zeros(dims,dims)
    if n<dims
        c[n+1,n+1]=1
    end
    return c
end
    
function equ_state(dims,n)
    c::Matrix{ComplexF64}=zeros(dims,dims)
    for i in 1:n
        c[i,i]=1/n    
    end
    return c
end
    
dim=4
η=0.084 
    
function a₋(n)      #n为维数
    s::Array{Complex{Float64}}=zeros(n,n)
    for i=1:n-1
        s[i,i+1]=sqrt(i)
    end
    return s
end

a₊(n)=adjoint(a₋(n));
a=a₊(dim)+a₋(dim)    
# a2=kron([0 0;1 0],exp(im*η*a))
# a3=kron([0 1;0 0],exp(-im*η*a))
a2=kron([0 0;1 0],im*η*a₊(dim))
a3=kron([0 1;0 0],-im*η*a₋(dim))   
#经过尝试发现,符合脉冲并不精准,都保留下来就很不精确
    
σ₊=[0 0;1 0]
σ₋=[0 1;0 0]
σ_x=[0 1;1 0]
σ_y=[0 -1im;1im 0]
σ_z=[1 0;0 -1]

function dm2vec(dm)
    return reshape(dm,:,1)
end

function vec2dm(vec)
    return reshape(vec,Int(sqrt(size(vec)[1])),:)
end
    
function temperature(nbar)
    return (6.626*1e-34*1e6*1)/(1.38*1e-23*log(1/nbar+1))
end
    
n=kron(I(2),a₊(dim)*a₋(dim))
en=kron([0 0;0 1],I(dim))

ν=1.
    
function H(Ω,Δ)
    unitary::Matrix{ComplexF64}=kron(I(dim*2),-im*(ν*n-Δ*en+Ω/2*a2+Ω'/2*a3))+kron(transpose(im*(ν*n-Δ*en+Ω/2*a2+Ω'/2*a3)),I(dim*2))
    return unitary
end
    
function Hv(Ω,Δ)
    unitary::Matrix{ComplexF64}=ν*n-Δ*en+Ω/2*a2+Ω'/2*a3
    return unitary
end


Hv (generic function with 1 method)

In [3]:
function R(t,phi,omega,delta)
    return exp(-im*t*Hv(omega*exp(im*phi),delta))
end

R (generic function with 1 method)

In [7]:
function swap_prod(dims,n)#蓝边带交换
    s=zeros(dim*2,dim*2)+I(dim*2)
    s[n+1,n+1]=0.;
    s[n+2+dims,n+2+dims]=0.;
    s[n+2+dims,n+1]=1.;
    s[n+1,n+2+dims]=1.;
    
    
    return s
end
swap=swap_prod(dim,2)

8×8 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0

In [8]:
function feval(x)
    swapfit=R(x[3],x[5],0.1,1)*R(x[2],x[4],0.1,1)*R(x[1],0,0.1,1)
    opres=norm(norm.(swapfit)-norm.(swap))
    println(round.(x,digits=5),"    ",round(opres,digits=4))
    return opres
end

feval (generic function with 1 method)

In [13]:
xini=[846.15742, 1087.60739, 1265.61274, 4.51198, 5.67969];
# xini=[470.00471, 1262.15604, 1247.92811, 4.53369, 6.77461] #from0
# xini=[682.05679, 1095.17636, 407.60225, 1.82914, 3.10828]#from1
# xini=[846.06595, 1088.08199, 1265.52912, 4.50001, 5.65826]#from2
lower=[0.,0,0,0,0];upper=[4*pi/η/0.1,4*pi/η/0.1,4*pi/η/0.1,2*pi,2*pi];
# res=optimize(feval,xini,ParticleSwarm(; lower = lower,upper = upper,n_particles = 10),Optim.Options(iterations = 10000))
res=optimize(feval,xini,LBFGS())
    

[846.16254, 1087.60739, 1265.61274, 4.51198, 5.67969]    0.2377
[846.1523, 1087.60739, 1265.61274, 4.51198, 5.67969]    0.2377
[846.15742, 1087.61398, 1265.61274, 4.51198, 5.67969]    0.2377
[846.15742, 1087.6008, 1265.61274, 4.51198, 5.67969]    0.2377
[846.15742, 1087.60739, 1265.6204, 4.51198, 5.67969]    0.2377
[846.15742, 1087.60739, 1265.60508, 4.51198, 5.67969]    0.2377
[846.15742, 1087.60739, 1265.61274, 4.51201, 5.67969]    0.2377
[846.15742, 1087.60739, 1265.61274, 4.51195, 5.67969]    0.2377
[846.15742, 1087.60739, 1265.61274, 4.51198, 5.67972]    0.2377
[846.15742, 1087.60739, 1265.61274, 4.51198, 5.67966]    0.2377
[846.15742, 1087.60739, 1265.61274, 4.51198, 5.67969]    0.2377
[846.16253, 1087.60743, 1265.61272, 4.50305, 5.67455]    0.2377
[846.15228, 1087.60743, 1265.61272, 4.50305, 5.67455]    0.2377
[846.15741, 1087.61402, 1265.61272, 4.50305, 5.67455]    0.2377
[846.15741, 1087.60084, 1265.61272, 4.50305, 5.67455]    0.2377
[846.15741, 1087.60743, 1265.62038, 4.50305

[846.05801, 1087.85237, 1265.7122, 4.50072, 5.66042]    0.2376
[846.05801, 1087.85237, 1265.7122, 4.50072, 5.66035]    0.2376
[846.05801, 1087.85237, 1265.7122, 4.50072, 5.66039]    0.2376
[846.06336, 1087.85201, 1265.71193, 4.50072, 5.66039]    0.2376
[846.05311, 1087.85201, 1265.71193, 4.50072, 5.66039]    0.2376
[846.05824, 1087.85859, 1265.71193, 4.50072, 5.66039]    0.2376
[846.05824, 1087.84542, 1265.71193, 4.50072, 5.66039]    0.2376
[846.05824, 1087.85201, 1265.71959, 4.50072, 5.66039]    0.2376
[846.05824, 1087.85201, 1265.70426, 4.50072, 5.66039]    0.2376
[846.05824, 1087.85201, 1265.71193, 4.50075, 5.66039]    0.2376
[846.05824, 1087.85201, 1265.71193, 4.50069, 5.66039]    0.2376
[846.05824, 1087.85201, 1265.71193, 4.50072, 5.66042]    0.2376
[846.05824, 1087.85201, 1265.71193, 4.50072, 5.66035]    0.2376
[846.05824, 1087.85201, 1265.71193, 4.50072, 5.66039]    0.2376
[846.06426, 1087.85054, 1265.71083, 4.50071, 5.6604]    0.2376
[846.05402, 1087.85054, 1265.71083, 4.50071,

[846.06609, 1088.0829, 1265.52867, 4.5, 5.65827]    0.2376
[846.06609, 1088.0829, 1265.52867, 4.5, 5.6582]    0.2376
[846.06609, 1088.0829, 1265.52867, 4.5, 5.65823]    0.2376
[846.07107, 1088.08199, 1265.52912, 4.50001, 5.65826]    0.2376
[846.06082, 1088.08199, 1265.52912, 4.50001, 5.65826]    0.2376
[846.06595, 1088.08858, 1265.52912, 4.50001, 5.65826]    0.2376
[846.06595, 1088.0754, 1265.52912, 4.50001, 5.65826]    0.2376
[846.06595, 1088.08199, 1265.53679, 4.50001, 5.65826]    0.2376
[846.06595, 1088.08199, 1265.52146, 4.50001, 5.65826]    0.2376
[846.06595, 1088.08199, 1265.52912, 4.50004, 5.65826]    0.2376
[846.06595, 1088.08199, 1265.52912, 4.49998, 5.65826]    0.2376
[846.06595, 1088.08199, 1265.52912, 4.50001, 5.65829]    0.2376
[846.06595, 1088.08199, 1265.52912, 4.50001, 5.65822]    0.2376
[846.06595, 1088.08199, 1265.52912, 4.50001, 5.65826]    0.2376


 * Status: success

 * Candidate solution
    Final objective value:     2.375502e-01

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 2.30e-04 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.82e-07 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.71e-11 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 7.18e-11 ≰ 0.0e+00
    |g(x)|                 = 2.14e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   1  (vs limit Inf)
    Iterations:    9
    f(x) calls:    36
    ∇f(x) calls:   36


In [14]:
xini

5-element Vector{Float64}:
  846.15742
 1087.60739
 1265.61274
    4.51198
    5.67969

In [15]:
res.minimizer

5-element Vector{Float64}:
  846.0659470067403
 1088.081987701874
 1265.5291231431522
    4.500007851039902
    5.658255672312758

In [16]:
res.minimum

0.2375501520154218

In [17]:
function feval2(x)
    swapfit=R(x[3],x[5],0.1,1)*R(x[2],x[4],0.1,1)*R(x[1],0,0.1,1)
    
    return swapfit
end

feval2 (generic function with 1 method)

In [18]:
round.(DataFrame(abs.(feval2(xini)),:auto),digits=4)

Row,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.9894,0.0,0.0,0.0,0.0,0.1454,0.0,0.0
2,0.0,0.9993,0.0,0.0,0.0,0.0,0.0379,0.0
3,0.0,0.0,0.0745,0.0,0.0,0.0,0.0,0.9972
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6,0.1454,0.0,0.0,0.0,0.0,0.9894,0.0,0.0
7,0.0,0.0379,0.0,0.0,0.0,0.0,0.9993,0.0
8,0.0,0.0,0.9972,0.0,0.0,0.0,0.0,0.0745


In [19]:
# round.(DataFrame(feval2(xini)/(feval2(xini)[1]/abs.(feval2(xini))[1]),:auto),digits=4)

In [20]:
round.(DataFrame(abs.(feval2(res.minimizer)),:auto),digits=4)

Row,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.9895,0.0,0.0,0.0,0.0,0.1446,0.0,0.0
2,0.0,0.9991,0.0,0.0,0.0,0.0,0.0418,0.0
3,0.0,0.0,0.0737,0.0,0.0,0.0,0.0,0.9973
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6,0.1446,0.0,0.0,0.0,0.0,0.9895,0.0,0.0
7,0.0,0.0418,0.0,0.0,0.0,0.0,0.9991,0.0
8,0.0,0.0,0.9973,0.0,0.0,0.0,0.0,0.0737
